In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __init__ import *
import numpy as np
from export.saver import read_pickle
from helpers.layout import Layout
from helpers.graph_helpers import *
from helpers.helpers import sort_and_group_objects
from typing import Dict
from domains.domain import Domain
from helpers.directions import get_axis, get_opposite_axis, Direction
from placement2.arrange import *
from placement2.place import *
from copy import copy, deepcopy
from icecream import ic
from visuals.plotter import plot_general
from itertools import product


In [3]:
layout: Layout = read_pickle("1001_amber_c_ag")
G = layout.graph
domains = layout.domains

In [5]:
create_placement_and_update_layout(layout)

arrange> NORTH EAST node added when processing 2: ['corridor']
arrange> NORTH EAST node added when processing 3: ['corridor']


Layout(shapes={'m_bed': <POLYGON ((2.13 -2.45, 2.13 0, 0 0, 0 -2.45, 2.13 -2.45))>, 'bed1': <POLYGON ((3.94 -2.45, 3.94 0, 2.13 0, 2.13 -2.45, 3.94 -2.45))>, 'm_closet': <POLYGON ((1.47 -3.33, 1.47 -2.45, 0 -2.45, 0 -3.33, 1.47 -3.33))>, 'transit': <POLYGON ((2.62 -3.41, 2.62 -2.45, 1.47 -2.45, 1.47 -3.41, 2.62 -3.41))>, 'bed1_closet': <POLYGON ((3.77 -2.82, 3.77 -2.45, 2.62 -2.45, 2.62 -2.82, 3.77 -2.82))>, 'm_bath': <POLYGON ((1.61 -4.57, 1.61 -3.33, 0 -3.33, 0 -4.57, 1.61 -4.57))>, 'corridor': <POLYGON ((2.06 -6.12, 2.06 -3.41, 1.61 -3.41, 1.61 -6.12, 2.06 -6.12))>, 'linen': <POLYGON ((2.63 -3.21, 2.63 -2.82, 2.06 -2.82, 2.06 -3.21, 2.63 -3.21))>, 'laundry': <POLYGON ((3.78 -3.91, 3.78 -2.82, 2.63 -2.82, 2.63 -3.91, 3.78 -3.91))>, 'bath': <POLYGON ((1.58 -5.5, 1.58 -4.57, 0 -4.57, 0 -5.5, 1.58 -5.5))>, 'den': <POLYGON ((3.39 -5.26, 3.39 -3.21, 1.58 -3.21, 1.58 -5.26, 3.39 -5.26))>, 'kitchen': <POLYGON ((1.47 -8.17, 1.47 -5.5, 0 -5.5, 0 -8.17, 1.47 -8.17))>, 'dining': <POLYGON ((3.01

In [5]:
arr = create_placement(layout)
arr

arrange> NORTH EAST node added when processing 2: ['corridor']
arrange> NORTH EAST node added when processing 3: ['corridor']


array([['m_bed', 'bed1', '', ''],
       ['m_closet', 'transit', 'bed1_closet', ''],
       ['m_bath', 'corridor', 'linen', 'laundry'],
       ['bath', 'corridor', 'den', ''],
       ['kitchen', 'dining', 'den_closet', ''],
       ['living', '', '', '']], dtype=object)

In [6]:
arr = remove_repeat_nodes(arr)
arr

array([['m_bed', 'bed1', '', ''],
       ['m_closet', 'transit', 'bed1_closet', ''],
       ['m_bath', 'corridor', 'linen', 'laundry'],
       ['bath', '', 'den', ''],
       ['kitchen', 'dining', 'den_closet', ''],
       ['living', '', '', '']], dtype=object)

In [7]:
darr = create_domains_arr(arr, domains)
darr

array([[Domain(m_bed, x=[0.11-2.24], y=[8.87-11.32]),
        Domain(bed1, x=[2.37-4.18], y=[8.87-11.32]), None, None],
       [Domain(m_closet, x=[0.11-1.58], y=[7.94-8.82]),
        Domain(transit, x=[1.79-2.94], y=[7.83-8.79]),
        Domain(bed1_closet, x=[3.03-4.18], y=[8.42-8.79]), None],
       [Domain(m_bath, x=[0.00-1.61], y=[6.49-7.73]),
        Domain(corridor, x=[1.79-2.24], y=[5.02-7.73]),
        Domain(linen, x=[2.37-2.94], y=[7.25-7.64]),
        Domain(laundry, x=[3.03-4.18], y=[7.25-8.34])],
       [Domain(bath, x=[0.03-1.61], y=[5.43-6.36]), None,
        Domain(den, x=[2.37-4.18], y=[5.12-7.17]), None],
       [Domain(kitchen, x=[0.11-1.58], y=[2.60-5.27]),
        Domain(dining, x=[1.73-3.27], y=[2.42-5.02]),
        Domain(den_closet, x=[3.38-4.18], y=[4.66-5.02]), None],
       [Domain(living, x=[0.11-3.27], y=[0.00-2.42]), None, None, None]],
      dtype=object)

In [16]:
get_north_node(darr, np.s_[3,2])

Domain(linen, x=[2.37-2.94], y=[7.25-7.64])

In [13]:
darr1 = place_nodes(darr)
darr1

first row, init col
first row, next col
next row, init col
next row, next col
next row, next col
next row, init col
next row, next col
next row, next col
next row, next col
next row, init col
next row, next col
next row, init col
next row, next col
next row, next col
next row, init col


array([[Domain(m_bed, x=[0.00-2.13], y=[-2.45-0.00]),
        Domain(bed1, x=[2.13-3.94], y=[-2.45-0.00]), None, None],
       [Domain(m_closet, x=[0.00-1.47], y=[-3.33--2.45]),
        Domain(transit, x=[1.47-2.62], y=[-3.41--2.45]),
        Domain(bed1_closet, x=[2.62-3.77], y=[-2.82--2.45]), None],
       [Domain(m_bath, x=[0.00-1.61], y=[-4.57--3.33]),
        Domain(corridor, x=[1.61-2.06], y=[-6.12--3.41]),
        Domain(linen, x=[2.06-2.63], y=[-3.21--2.82]),
        Domain(laundry, x=[2.63-3.78], y=[-3.91--2.82])],
       [Domain(bath, x=[0.00-1.58], y=[-5.50--4.57]), None,
        Domain(den, x=[1.58-3.39], y=[-5.26--3.21]), None],
       [Domain(kitchen, x=[0.00-1.47], y=[-8.17--5.50]),
        Domain(dining, x=[1.47-3.01], y=[-8.10--5.50]),
        Domain(den_closet, x=[3.01-3.81], y=[-5.62--5.26]), None],
       [Domain(living, x=[0.00-3.16], y=[-10.59--8.17]), None, None,
        None]], dtype=object)

In [14]:
new_domains = create_new_domains(darr1)
new_domains
# have to be prescriptive about how overwrite corridor

{'m_bed': Domain(m_bed, x=[0.00-2.13], y=[-2.45-0.00]),
 'bed1': Domain(bed1, x=[2.13-3.94], y=[-2.45-0.00]),
 'm_closet': Domain(m_closet, x=[0.00-1.47], y=[-3.33--2.45]),
 'transit': Domain(transit, x=[1.47-2.62], y=[-3.41--2.45]),
 'bed1_closet': Domain(bed1_closet, x=[2.62-3.77], y=[-2.82--2.45]),
 'm_bath': Domain(m_bath, x=[0.00-1.61], y=[-4.57--3.33]),
 'corridor': Domain(corridor, x=[1.61-2.06], y=[-6.12--3.41]),
 'linen': Domain(linen, x=[2.06-2.63], y=[-3.21--2.82]),
 'laundry': Domain(laundry, x=[2.63-3.78], y=[-3.91--2.82]),
 'bath': Domain(bath, x=[0.00-1.58], y=[-5.50--4.57]),
 'den': Domain(den, x=[1.58-3.39], y=[-5.26--3.21]),
 'kitchen': Domain(kitchen, x=[0.00-1.47], y=[-8.17--5.50]),
 'dining': Domain(dining, x=[1.47-3.01], y=[-8.10--5.50]),
 'den_closet': Domain(den_closet, x=[3.01-3.81], y=[-5.62--5.26]),
 'living': Domain(living, x=[0.00-3.16], y=[-10.59--8.17])}

In [15]:
plot_general(new_domains)